In [ ]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score 
from prefect import flow, task
from joblib import load, dump
from tqdm import tqdm
import pandas as pd 
import joblib
import mlflow
import datetime
import requests

In [37]:
df = pd.read_csv('/home/mussie/Music/home projects/nice_one/seving-ml-model-using-fastapi-and-docker/target.csv')
df = df.drop_duplicates()
df = df.drop(columns=['customer_id','country','gender'],axis =1)
    
train_data = df[:7000]
val_data = df[7000:]


target = "churn"
num_features = ['credit_score', 'age', 'tenure', 'balance', 'products_number','credit_card', 'active_member', 'estimated_salary']
cat_features =[]
    
X = train_data.drop('churn', axis=1)
y = train_data['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

    # Print the shapes of the arrays
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)
    

Shape of X_train: (5600, 8)
Shape of y_train: (5600,)
Shape of X_test: (1400, 8)
Shape of y_test: (1400,)


In [38]:
 # Train DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=1)

dt_param_grid = {
        'max_depth': [3, 5, 7, 9, 11, 13],
        'criterion': ['gini', 'entropy']
    }

dt_gs = GridSearchCV(
        estimator=dt,
        param_grid=dt_param_grid,
        cv=5,
        n_jobs=1,
        scoring='accuracy',
        verbose=0
    )
dt_model = dt_gs.fit(X_train, y_train)

In [39]:
# Train prediction
y_train_pred = dt_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)

# Test prediction
y_test_pred = dt_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.8573214285714286
Test Accuracy: 0.8521428571428571


In [40]:
# Train MAE
train_mae = mean_absolute_error(y_train, y_train_pred)
print("Train MAE:", train_mae)

# Test MAE
test_mae = mean_absolute_error(y_test, y_test_pred)
print("Test MAE:", test_mae)

Train MAE: 0.14267857142857143
Test MAE: 0.14785714285714285


In [41]:
import joblib

# Assuming 'dt_model' is your trained Decision Tree model
model_path = '/home/mussie/Music/home projects/nice_one/seving-ml-model-using-fastapi-and-docker/model.joblib'

with open(model_path, 'wb') as f_out:
    joblib.dump(dt_model, f_out)


In [43]:
data = '/home/mussie/Music/home projects/nice_one/seving-ml-model-using-fastapi-and-docker/data'
val_data.to_parquet('data/reference.csv')

In [ ]:
column_mapping = ColumnMapping(
    target=None,
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features
)